# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()


normalized_histos = list()

Welcome to JupyROOT 6.24/06


In [2]:
# %load python/drawingTools.py


In [12]:
# %load samples.py

samples = [
# #     HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v150A.70B', 'ref'),
# #     HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v151A.80El', '12.3'),
#     HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v151A.80Fl', 'ref'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152B.90A', '12p5', 'CMSSW 12.5'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152B.91E', 'compID-float'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152D.91G', 'compID', 'CMSSW 12.5+PR'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152B.91C', 'ellID'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152D.91G', 'compID', 'CMSSW 12.5+PR'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152F.91G', 'menu', 'menu'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v152F.91Gi', 'old', 'old'),
    HistoFile('nugun_alleta_pu200_rate', 'PU200', 'v160A.91Gi', 'new', 'new'),


]


In [4]:
for hf in samples:
    hf.print_primitives()

<HistoFile ../plots/histos_nugun_alleta_pu200_rate_v152B.90A.root, 12p5>
--- HistoClass: 
['RateHistos']
--- TPs: 
[   'TkEmL2',
    'TkEleEE',
    'TkEmEB',
    'EGStaEE',
    'TkEleL2',
    'TkEleEB',
    'TkEmEE',
    'EGStaEB']
--- TP selections:
{   'EGStaEB': [   'IDTightP',
                   'IDTightE',
                   'EtaF',
                   'IDTightPEtaF',
                   'all',
                   'IDTightEEtaF'],
    'EGStaEE': ['IDTightSEtaABC', 'EtaABCD', 'IDTightSEtaABCD', 'EtaABC'],
    'TkEleEB': [   'IDTightEIso0p1',
                   'Iso0p1EtaF',
                   'IDTightPIso0p3EtaF',
                   'IDTightP',
                   'IDTightEIso0p2',
                   'IDTightPIso0p1',
                   'Iso0p1',
                   'IDTightEIso0p2EtaF',
                   'Iso0p2EtaF',
                   'EtaF',
                   'IDTightEIso0p1EtaF',
                   'Iso0p2',
                   'IDTightPIso0p1EtaF',
                   'IDTightEIso

    'TkEleEB': [   'IDTightEIso0p1',
                   'Iso0p1EtaF',
                   'IDTightPIso0p3EtaF',
                   'IDTightP',
                   'IDTightEIso0p2',
                   'IDTightPIso0p1',
                   'Iso0p1',
                   'IDTightEIso0p2EtaF',
                   'Iso0p2EtaF',
                   'EtaF',
                   'IDTightEIso0p1EtaF',
                   'Iso0p2',
                   'IDTightPIso0p1EtaF',
                   'IDTightEIso0p3',
                   'Iso0p3',
                   'IDTightE',
                   'IDTightPIso0p2EtaF',
                   'IDTightPEtaF',
                   'IDTightPIso0p3',
                   'all',
                   'IDTightPIso0p2',
                   'IDTightEEtaF',
                   'IDTightEIso0p3EtaF',
                   'Iso0p3EtaF'],
    'TkEleEE': [   'IDCompWP800EtaABC',
                   'IDCompWP800EtaBC',
                   'IDCompWP955EtaBC',
                   'IDTightEEtaBC',
      

In [13]:
import python.collections as collections
import python.selections as selections
from cfg import *

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



# #  ==== labels ===============================================
# tp_labels = histo_primitives[['data', 'data_label']].drop_duplicates().set_index('data').T.to_dict('records')[0]
# tp_selection_labels = histo_primitives[['data_sel', 'data_sel_label']].drop_duplicates().set_index('data_sel').T.to_dict('records')[0]
# gen_selection_labels = histo_primitives[['gen_sel', 'gen_sel_label']].drop_duplicates().set_index('gen_sel').T.to_dict('records')[0]






In [6]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [14]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.RateHistos)


--- RateHistos
# of plots: 462
# of primitives: 154
--- RateHistos
# of plots: 462
# of primitives: 154
--- RateHistos
# of plots: 702
# of primitives: 234
--- RateHistos
# of plots: 462
# of primitives: 154
--- RateHistos
# of plots: 702
# of primitives: 234
--- RateHistos
# of plots: 90
# of primitives: 30
--- RateHistos
# of plots: 852
# of primitives: 284
--- RateHistos
# of plots: 852
# of primitives: 284
CPU times: user 399 ms, sys: 27.1 ms, total: 426 ms
Wall time: 428 ms


In [8]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,12p5,PU200,TkEleEE,Iso0p3EtaABC,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b72c01f0>
1,12p5,PU200,TkEmL2,IDTightPIso0p1EtaFABC,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b72a3bb0>
2,12p5,PU200,TkEleL2,Iso0p1EtaABC,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b72a3e80>
3,12p5,PU200,TkEleL2,IDTightEIso0p1EtaF,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b72a3f70>
4,12p5,PU200,TkEmEB,EtaF,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b72a3af0>
...,...,...,...,...,...,...,...
1289,new,PU200,TkEmEB,IDTightPIsoPhoEBEtaF,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b7385300>
1290,new,PU200,EGStaEE,EtaABC,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b7385360>
1291,new,PU200,TkEleL2,IDTightPIsoEleEEEtaF,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b73853c0>
1292,new,PU200,TkEmEB,IDTightEIso0p3EtaF,None,<class 'python.l1THistos.RateHistos'>,<drawingTools.HProxy object at 0x2b7385420>


In [9]:
hplot.data['sample'].unique()

array(['12p5', 'compID-float', 'ellID', 'compID', 'menu', 'old', 'new'],
      dtype=object)

In [10]:
hplot.data['tp'].unique()

array(['TkEleEE', 'TkEmL2', 'TkEleL2', 'TkEmEB', 'TkEmEE', 'EGStaEB',
       'TkEleEB', 'EGStaEE', 'TkEleL2Ell', 'TkEleEllEE', 'TkEmL2Ell'],
      dtype=object)

In [11]:
hplot.data[hplot.data.tp == 'tkEleEE'].tp_sel.unique()

array([], dtype=object)

In [9]:
# %load python/rate_utilities.py
import ROOT


def convertRateToGraph(histo_rate, params=None):
    """
    Converts a rate TH1F to a graph.
    Optionally applyes a linear scale mapping to the X axis.
    """
    
    xaxis = histo_rate.GetXaxis()
    fNpoints = xaxis.GetNbins()
    fX = [0]*fNpoints
    fEX = [0]*fNpoints
    fY = [0]*fNpoints
    fEY = [0]*fNpoints

    graph = ROOT.TGraphAsymmErrors()
    graph.Set(fNpoints)
    for i in range(0, fNpoints):
        if params is None:
            fX[i] = xaxis.GetBinLowEdge(i + 1)
        else:
            fX[i] = params[0] + params[1]*xaxis.GetBinLowEdge(i + 1)
        fY[i] = histo_rate.GetBinContent(i + 1)
        fEX[i] = histo_rate.GetBinWidth(i + 1) * ROOT.gStyle.GetErrorX()
        fEY[i] = histo_rate.GetBinError(i + 1)
    return fX, fY, fEX, fEX, fEY, fEY


def cutAtRate(histo, rate):
    """
    Determines the X value (pt cut) corresponding to Y value (rate)
    """
    cut = None
    for ix in range(1, histo.GetNbinsX()+1):
        if histo.GetBinContent(ix) <= rate:
            cut = histo.GetXaxis().GetBinLowEdge(ix)
            break
    return cut


In [10]:
import os
wc = wp.WebPageCreator(
    topic_dir='rate', 
    project_dir='V91-CMSSW12.5-menu', 
    base_dir='/Users/cerminar/CERNbox/www/plots/', 
    tmp_dir=os.environ['TMPDIR'])

In [8]:
newconfig=rleg_config
# newconfig=tdr_config

newconfig.marker_size = 1
newconfig.legend_size=(0.5, 0.3)
newconfig.legend_position=(0.12, 0.15)
newconfig.marker_styles.append(10)

In [50]:
# %load ./validation_rate.py

smps = ['12p5', 'compID']

# 
configs = [
    (['TkEmEE'],  ['IDTightPEtaABC'],       'hRate_TkEmEE_IDTightP_EtaABC'),
    (['TkEmEE'],  ['IDTightPIso0p1EtaABC'], 'hRate_TkEmEE_IDTightPIso0p1_EtaABC'),
    (['TkEmEE'],  ['EtaABC'],               'hRate_TkEmEE_IDLooseP_EtaABC'),
    (['TkEmEB'],  ['EtaF'],                 'hRate_TkEmEB_IDLooseP_EtaF'),

    (['TkEleEE'], ['IDTightEEtaABC'],       'hRate_TkEleEE_IDTightE_EtaABC'),
    (['TkEleEE'], ['EtaABC'],               'hRate_TkEleEE_IDLooseE_EtaABC'),
    (['TkEleEB'], ['EtaF'],                 'hRate_TkEleEB_IDLooseE_EtaF'),
    (['TkEleEB'], ['IDTightEEtaF'],         'hRate_TkEleEB_IDTightE_EtaF'),
    (['TkEleEB'], ['IDTightEIso0p1EtaF'],   'hRate_TkEleEB_IDTightEIso0p1_EtaF'),

    (['TkEmL2'],  ['EtaABC'],               'hRate_TkEmL2_IDLooseP_EtaABC'),
    (['TkEmL2'],  ['IDTightPEtaABC'],       'hRate_TkEmL2_IDTightP_EtaABC'),
    (['TkEmL2'],  ['EtaF'],                 'hRate_TkEmEB_IDLooseP_EtaF'),

    (['TkEleL2'], ['IDTightEEtaABC'],       'hRate_TkEleL2_IDTightE_EtaABC'),
    (['TkEleL2'], ['EtaABC'],               'hRate_TkEleL2_IDLooseE_EtaABC'),
    (['TkEleL2'], ['EtaF'],                 'hRate_TkEleL2_IDLooseE_EtaF'),
    (['TkEleL2'], ['IDTightEEtaF'],         'hRate_TkEleL2_IDTightE_EtaF'),

    (['TkEleL2'], ['IDTightEEtaFABC'],       'hRate_TkEleL2_IDTightE_EtaFABC'),
    (['TkEleL2'], ['EtaFABC'],               'hRate_TkEleL2_IDLooseE_EtaFABC'),

]

for objs, objs_sel, h_name in configs:
    print(f'obj: {objs}, sel: {objs_sel}, histo: {h_name}')

    dm = DrawMachine(newconfig)
    dm.config.legend_position = (0.4, 0.4)

    hsets, labels, text = hplot.get_histo(histos.RateHistos, smps, 'PU200', objs, objs_sel, None)
    dm.addHistos([his.h_pt for his in hsets], labels=labels)

    dm.addRatioHisto(1,0)
#     dm.addRatioHisto(2,0)

    dm.draw(
        text=text,
        y_min=0.1, y_max=40000,
        x_min=0, x_max=60,
        y_min_ratio=0.5, y_max_ratio=2.0,
        y_log=True, 
        x_axis_label='online p_{T} thresh. [GeV]',
        h_lines=[20,100,1000],
    do_ratio=True)
    dm.toWeb(name=h_name, page_creator=wc)

#




#wc.publish()

obj: ['TkEmEE'], sel: ['IDTightPEtaABC'], histo: hRate_TkEmEE_IDTightP_EtaABC
obj: ['TkEmEE'], sel: ['IDTightPIso0p1EtaABC'], histo: hRate_TkEmEE_IDTightPIso0p1_EtaABC
obj: ['TkEmEE'], sel: ['EtaABC'], histo: hRate_TkEmEE_IDLooseP_EtaABC
obj: ['TkEmEB'], sel: ['EtaF'], histo: hRate_TkEmEB_IDLooseP_EtaF
obj: ['TkEleEE'], sel: ['IDTightEEtaABC'], histo: hRate_TkEleEE_IDTightE_EtaABC
obj: ['TkEleEE'], sel: ['EtaABC'], histo: hRate_TkEleEE_IDLooseE_EtaABC
obj: ['TkEleEB'], sel: ['EtaF'], histo: hRate_TkEleEB_IDLooseE_EtaF
obj: ['TkEleEB'], sel: ['IDTightEEtaF'], histo: hRate_TkEleEB_IDTightE_EtaF
obj: ['TkEleEB'], sel: ['IDTightEIso0p1EtaF'], histo: hRate_TkEleEB_IDTightEIso0p1_EtaF
obj: ['TkEmL2'], sel: ['EtaABC'], histo: hRate_TkEmL2_IDLooseP_EtaABC
obj: ['TkEmL2'], sel: ['IDTightPEtaABC'], histo: hRate_TkEmL2_IDTightP_EtaABC
obj: ['TkEmL2'], sel: ['EtaF'], histo: hRate_TkEmEB_IDLooseP_EtaF
[WebPageCreator]***Warning: overwriting canvas: hRate_TkEmEB_IDLooseP_EtaF!
obj: ['TkEleL2'], sel:

Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDTightP_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDTightP_EtaABC.pdf has been created
Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDTightPIso0p1_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDTightPIso0p1_EtaABC.pdf has been created
Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDLooseP_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDLooseP_EtaABC.pdf has been created
Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6

In [52]:

wc.publish()

publishing canvas: hRate_TkEmEE_IDTightP_EtaABC
publishing canvas: hRate_TkEmEE_IDTightPIso0p1_EtaABC
publishing canvas: hRate_TkEmEE_IDLooseP_EtaABC
publishing canvas: hRate_TkEmEB_IDLooseP_EtaF
publishing canvas: hRate_TkEleEE_IDTightE_EtaABC
publishing canvas: hRate_TkEleEE_IDLooseE_EtaABC
publishing canvas: hRate_TkEleEB_IDLooseE_EtaF
publishing canvas: hRate_TkEleEB_IDTightE_EtaF
publishing canvas: hRate_TkEleEB_IDTightEIso0p1_EtaF
publishing canvas: hRate_TkEmL2_IDLooseP_EtaABC
publishing canvas: hRate_TkEmL2_IDTightP_EtaABC
publishing canvas: hRate_TkEleL2_IDTightE_EtaABC
publishing canvas: hRate_TkEleL2_IDLooseE_EtaABC
publishing canvas: hRate_TkEleL2_IDLooseE_EtaF
publishing canvas: hRate_TkEleL2_IDTightE_EtaF
publishing canvas: hRate_TkEleL2_IDTightE_EtaFABC
publishing canvas: hRate_TkEleL2_IDLooseE_EtaFABC


In [ ]:
--- TP selections:
{   'TkEleL2': [   'MenuEleIsoTightEtaEB',
                   'MenuEleLoose',
                   'MenuEleLooseEtaEB',
                   'MenuEleIsoLooseEtaEB',
                   'MenuEleTight',
                   'MenuEleIsoLoose',
                   'MenuEleIsoTight',
                   'MenuEleTightEtaEB'],
    'TkEleL2Ell': [   'MenuEleTightEtaEB',
                      'MenuEleIsoLooseEtaEB',
                      'MenuEleIsoTightEtaEB',
                      'MenuEleLoose',
                      'MenuEleTight',
                      'MenuEleIsoLoose',
                      'MenuEleLooseEtaEB',
                      'MenuEleIsoTight'],
     'TkEmL2': ['MenuPhoIso', 'MenuPhoIsoEtaEB'],
    'TkEmL2Ell': ['MenuPhoIsoEtaEB', 'MenuPhoIso']}

In [15]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['old', 'new'], 'PU200', ['TkEleL2'], ['IDTightEEtaFABC'], None)
dm.addHistos([his.h_norm for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEm_MenuPhoIso_EtaAll', page_creator=wc)



 .      # ev: 1000.0
normalize to 31000
 .      # ev: 1000.0
normalize to 31000


In [16]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['old', 'new'], 'PU200', ['TkEleL2'], ['IDTightEEtaFABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEm_MenuPhoIso_EtaAll', page_creator=wc)



In [92]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEmL2', 'TkEmL2Ell'], ['MenuPhoIso'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEm_MenuPhoIso_EtaAll', page_creator=wc)



Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5-menu/rate/hRate_TkEm_MenuPhoIso_EtaAll.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5-menu/rate/hRate_TkEm_MenuPhoIso_EtaAll.pdf has been created


In [93]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEmL2'], ['MenuPhoIso', 'MenuPhoIsoEtaEB', 'MenuPhoIsoEtaEE'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.1, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEm_MenuPhoIso_Eta', page_creator=wc)



Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5-menu/rate/hRate_TkEm_MenuPhoIso_Eta.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5-menu/rate/hRate_TkEm_MenuPhoIso_Eta.pdf has been created


In [67]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', 'TkEleL2Ell'], ['MenuEleTight'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [68]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', 'TkEleL2Ell'], ['MenuEleLoose'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [94]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2'], ['MenuEleTight', 'MenuEleTightEtaEB', 'MenuEleTightEtaEE'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [91]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2'], ['MenuEleTight', 'MenuEleLoose',], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2Ell'], ['MenuEleTight'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['Ell ID'])


dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [71]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', 'TkEleL2Ell'], ['MenuEleIsoTightEtaEE', 'MenuEleIsoLooseEtaEE',], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


In [76]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', ], ['MenuEleIsoTight', 'MenuEleTight','MenuEleIsoLoose'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [88]:

dm = DrawMachine(newconfig)


hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', ], ['MenuEleIsoLoose', 'MenuEleIsoTight'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2Ell', ], ['MenuEleIsoTight'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['Ell ID (tight ID)'])

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.8, y_max_ratio=1.2,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [84]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', ], ['MenuEleIsoLooseEtaEB', 'MenuEleIsoTightEtaEB'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2Ell', ], ['MenuEleIsoTightEtaEB'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['Ell ID (tight ID)'])


dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.8, y_max_ratio=1.2,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [87]:

dm = DrawMachine(newconfig)


hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2', ], ['MenuEleIsoLooseEtaEE', 'MenuEleIsoTightEtaEE'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['menu'], 'PU200', ['TkEleL2Ell', ], ['MenuEleIsoTightEtaEE'], None)
dm.addHistos([his.h_pt for his in hsets], labels=['Ell ID (tight ID)'])


dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)

dm.config.legend_position = (0.4, 0.4)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.1, y_max_ratio=2,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightP_EtaABC', page_creator=wc)



In [60]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEmEE'], ['IDTightPIso0p1EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)

dm.config.legend_position=(0.4, 0.5)
dm.draw(
    text=text,
    y_min=0.1, y_max=5000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_IDTightPIso0p1_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000
***Warning: Ratio plot can not be added: list index out of range
***Warning: Ratio plot can not be added: list index out of range


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDTightPIso0p1_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_IDTightPIso0p1_EtaABC.pdf has been created


In [61]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEmEE'], ['EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)


dm.config.legend_position=(0.4, 0.5)
dm.draw(
    text=text,
    y_min=1, y_max=10000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEE_all_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000
***Warning: Ratio plot can not be added: list index out of range


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_all_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_all_EtaABC.pdf has been created


In [62]:
dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEmEE'], ['EGq3EtaABCIso0p1','IDTightPIso0p1EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEmEE'], ['Iso0p1EtaABC'], None)
# dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)


dm.config.legend_position=(0.4, 0.5)
dm.draw(
    text=text,
    y_min=1, y_max=10000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True,
    y_min_ratio=0.8, 
    y_max_ratio=2)


# dm.toWeb(name='hRate_TkEmEE_Iso0p1_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_Iso0p1_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEE_Iso0p1_EtaABC.pdf has been created


In [63]:

dm = DrawMachine(newconfig)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEmEB'], ['EtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)
dm.config.legend_position=(0.3, 0.5)
dm.draw(
    text=text,
    y_min=1, y_max=70000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmEB_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEB_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmEB_EtaF.pdf has been created


In [20]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleEE'], ['IDTightEEtaABC', 'EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)





normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


In [53]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEleEE'], ['EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['compID'], 'PU200', ['TkEleEllEE'], ['EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleEE_EllID_IDLooseE_EtaABC', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_EllID_IDLooseE_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_EllID_IDLooseE_EtaABC.pdf has been created


In [54]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEleEE'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['compID'], 'PU200', ['TkEleEllEE'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleEE_Ell_IDTightE_EtaABC', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_Ell_IDTightE_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_Ell_IDTightE_EtaABC.pdf has been created


In [66]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleEE'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=4000,
    x_min=0, x_max=60,
    y_min_ratio=0.1, y_max_ratio=2,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleEE_IDTightE_EtaABC', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDTightE_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDTightE_EtaABC.pdf has been created


In [55]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)



hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleEE'], ['IDTightEEtaABC', 'IDCompWP955EtaABC', 'IDCompWP900EtaABC', 'IDCompWP800EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=4000,
    x_min=0, x_max=60,
    y_min_ratio=0.0, y_max_ratio=2.1,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[9],
    h_lines_ratio=[0.5, 1.5],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleEE_IDCompWP_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDCompWP_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDCompWP_EtaABC.pdf has been created


In [37]:
newconfig.marker_styles

[20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 10]

In [25]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['compID-new'], 'PU200', ['TkEleEE'], ['IDTightEEtaABC', 'IDTightEEtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=1000,
    x_min=0, x_max=60,
    y_min_ratio=0.1, y_max_ratio=2,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleEE_IDTightE_EtaABC', page_creator=wc)



[WebPageCreator]***Warning: overwriting canvas: hRate_TkEleEE_IDTightE_EtaABC!


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDTightE_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDTightE_EtaABC.pdf has been created


In [107]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['ellID', 'compID'], 'PU200', ['TkEleEE'], ['EGq3EtaABC', 'IDTightEEtaABC', 'IDCompWP995EtaABC', 'IDCompWP955EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=1000,
    x_min=0, x_max=60,
    y_min_ratio=0.1, y_max_ratio=2,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleEE_IDTightE_CompIDWP_EtaABC', page_creator=wc)



Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDTightE_CompIDWP_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEE_IDTightE_CompIDWP_EtaABC.pdf has been created


In [37]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['old-ID', '12p3','12p5','12p5new'], 'PU200', ['TkEleEE'], ['EGq3EtaABC', 'IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p3','12p5','12p5new'], 'PU200', ['TkEleEE'], ['EtaABC'], None)
# dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)

dm.draw(
    text=text,
    y_min=1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleEE_all_EtaABC', page_creator=wc)



normalize to 31000
***Warning: Ratio plot can not be added: list index out of range


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V90-CMSSW12.5/rate/hRate_TkEleEE_all_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V90-CMSSW12.5/rate/hRate_TkEleEE_all_EtaABC.pdf has been created


In [70]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleEB'], ['EtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleEB_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEB_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEB_EtaF.pdf has been created


In [71]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleEB'], ['IDTightEEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=60,
    y_min_ratio=0.5, y_max_ratio=1.5,

    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleEB_TightID_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEB_TightID_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleEB_TightID_EtaF.pdf has been created


In [40]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p3','12p5new'], 'PU200', ['TkEleEB'], ['LooseTkIDEtaFIso0p1', 'IDTightEIso0p1EtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleEB_TightIDIso0p1_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V90-CMSSW12.5/rate/hRate_TkEleEB_TightIDIso0p1_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V90-CMSSW12.5/rate/hRate_TkEleEB_TightIDIso0p1_EtaF.pdf has been created


In [72]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleL2'], ['IDTightEEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaF.pdf has been created


In [73]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['ellID', 'compID'], 'PU200', ['TkEleL2'], ['IDTightEEtaFABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0., y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaFABC', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaFABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaFABC.pdf has been created


In [143]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p3', '12p5new'], 'PU200', ['TkEleL2'], ['IDTightEEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.8, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V90-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V90-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaF.pdf has been created


In [74]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEleL2'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=0.1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.8, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaABC.pdf has been created


In [75]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEmL2'], ['IDTightPEtaFABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.8, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmL2_TightID_EtaFABC', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaFABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaFABC.pdf has been created


In [76]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5','compID'], 'PU200', ['TkEmL2'], ['EtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.8, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmL2_EtaF', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmL2_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmL2_EtaF.pdf has been created


In [77]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', 'compID'], 'PU200', ['TkEmL2'], ['IDTightPEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.5, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEmL2_TightID_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaABC.pdf has been created


In [78]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5new', 'compID'], 'PU200', ['TkEleL2'], ['IDTightEIso0p1EtaFABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.8, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaFABC', page_creator=wc)



normalize to 31000
***Warning: Ratio plot can not be added: list index out of range
[WebPageCreator]***Warning: overwriting canvas: hRate_TkEleL2_TightID_EtaFABC!


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaFABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V91-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaFABC.pdf has been created


In [56]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEleEB','TkEleL2'], ['IDTightEEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaF_new', page_creator=wc)



Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaF_new.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaF_new.pdf has been created


In [58]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaABC', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaABC.pdf has been created


In [59]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


# dm.toWeb(name='hRate_TkEleL2_TightID_EtaABC_new', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaABC_new.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightID_EtaABC_new.pdf has been created


In [60]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEIso0p2EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleL2_TightIDIso0p1_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightIDIso0p1_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightIDIso0p1_EtaABC.pdf has been created


In [61]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEIso0p2EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleL2_TightIDIso0p1_EtaABCnew', page_creator=wc)



normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightIDIso0p1_EtaABCnew.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightIDIso0p1_EtaABCnew.pdf has been created


In [62]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5', '12p5new'], 'PU200', ['TkEleL2', 'TkEleEE'], ['IDTightEIso0p1EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=4000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEleL2_TightIDIso0p1_EtaABC', page_creator=wc)



normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
[WebPageCreator]***Warning: overwriting canvas: hRate_TkEleL2_TightIDIso0p1_EtaABC!


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightIDIso0p1_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEleL2_TightIDIso0p1_EtaABC.pdf has been created


In [63]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEmEE','TkEmL2'], ['IDTightPEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=10000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEmL2_TightID_EtaABC', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaABC.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaABC.pdf has been created


In [64]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEmEE','TkEmL2'], ['IDTightPEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=10000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEmL2_TightID_EtaABC_new', page_creator=wc)



normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaABC_new.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaABC_new.pdf has been created


In [67]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.7

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEmEB','TkEmL2'], ['IDTightPEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=50000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEmL2_TightID_EtaF', page_creator=wc)



[WebPageCreator]***Warning: overwriting canvas: hRate_TkEmL2_TightID_EtaF!


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightID_EtaF.pdf has been created


In [31]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.7

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEmEB','TkEmL2'], ['IDTightPIso0p1EtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=50000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEmL2_TightIDIso0p1_EtaF', page_creator=wc)


 .      # ev: 100000.0
normalize to 31000
 .      # ev: 100000.0
normalize to 31000


Info in <TCanvas::Print>: png file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightIDIso0p1_EtaF.png has been created
Info in <TCanvas::Print>: pdf file /var/folders/8z/0424tdz97zsc8klvdy6_xl5r0000gn/T/V85-CMSSW12.5/rate/hRate_TkEmL2_TightIDIso0p1_EtaF.pdf has been created


In [56]:
wc.publish()

Yes/No: yes
publishing canvas: hRate_TkEmEE_IDTightP_EtaABC
publishing canvas: hRate_TkEmEE_IDTightPIso0p1_EtaABC
publishing canvas: hRate_TkEmEE_IDLooseP_EtaABC
publishing canvas: hRate_TkEmEB_IDLooseP_EtaF
publishing canvas: hRate_TkEleEE_IDTightE_EtaABC
publishing canvas: hRate_TkEleEE_IDLooseE_EtaABC
publishing canvas: hRate_TkEleEB_IDLooseE_EtaF
publishing canvas: hRate_TkEleEB_IDTightE_EtaF
publishing canvas: hRate_TkEleEB_IDTightEIso0p1_EtaF
publishing canvas: hRate_TkEmL2_IDLooseP_EtaABC
publishing canvas: hRate_TkEmL2_IDTightP_EtaABC
publishing canvas: hRate_TkEleL2_IDTightE_EtaABC
publishing canvas: hRate_TkEleL2_IDLooseE_EtaABC
publishing canvas: hRate_TkEleL2_IDLooseE_EtaF
publishing canvas: hRate_TkEleL2_IDTightE_EtaF
publishing canvas: hRate_TkEleL2_IDTightE_EtaFABC
publishing canvas: hRate_TkEleL2_IDLooseE_EtaFABC
publishing canvas: hRate_TkEleEE_EllID_IDLooseE_EtaABC
publishing canvas: hRate_TkEleEE_Ell_IDTightE_EtaABC
publishing canvas: hRate_TkEleEE_IDCompWP_EtaABC


In [28]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

# hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE', 'TkEmEE'], ['EGq3EtaBC'], None)
# dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['quality', 'ref'], 'PU200', ['PFNFtkEmEE', 'TkEmEE', 'TkEmL2'], ['EGq3EtaBC', 'IDTightPEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    
   do_ratio=True)




normalize to 31000


In [29]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEmEE', 'TkEmL2'], ['IDTightPEtaABC', 'EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)


dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True,
    y_min_ratio=0.7,
    y_max_ratio=3.)




No match found for: sample: quality pu: PU200, tp: ['TkEmEE', 'TkEmL2'], tp_sel: ['IDTightPEtaABC', 'EtaABC'], gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>


TypeError: 'NoneType' object is not iterable

In [ ]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEmEE', 'TkEmEE'], ['EGq1EtaBCD'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




In [ ]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['TkEleEE'], ['EGq3EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=1, y_max=1000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




In [ ]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEEtaABC', 'EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)
dm.addRatioHisto(2,0)
dm.addRatioHisto(3,0)

dm.draw(
    text=text,
    y_min=1, y_max=1000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
    do_ratio=True,
    y_min_ratio=0.7,
    y_max_ratio=3.)




In [51]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEE','TkEleEE'], ['EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=1000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [122]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['quality', 'ref'], 'PU200', ['TkEleEB'], ['all', 'LooseTkID', 'IDTightE'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=10, y_max=60000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=False)




In [53]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEmEB'], ['all', 'EtaF', 'IDTightP'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEmL2'], ['EtaF', 'IDTightPEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=10, y_max=60000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [54]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEB','TkEleEB'], ['all', 'LooseTkID'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEleEB', 'TkEleL2'], ['EtaF','IDTightEEtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)



dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=1, y_max=8000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [55]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEleEB'], ['EtaF', 'IDTightEEtaF', 'IDTightEIso0p1EtaF', 'Iso0p1EtaF'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)



dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=1, y_max=8000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [56]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)


hsets, labels, text = hplot.get_histo(histos.RateHistos, 'quality', 'PU200', ['TkEleEE'], ['EtaABC', 'IDTightEEtaABC', 'IDTightEIso0p1EtaABC', 'Iso0p1EtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)



dm.addRatioHisto(0,2)
dm.addRatioHisto(1,2)
dm.addRatioHisto(3,2)

dm.draw(
    text=text,
    y_min=1, y_max=8000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
0.0 60.0
0.0 60.0
0.0 60.0


In [57]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['PFNFtkEleEB','TkEleEB'], ['LooseTkID'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=8000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




0.0 60.0
0.0 60.0
0.0 60.0


In [58]:
   'HGCCl3d': [   'EgBdtHE975PUIdEtaBCD',
                   'EmEtaABC',
                   'PUIdEtaBCD',
                   'EmPUId',
                   'EgBdtLE975',
                   'EtaBC',
                   'EgBdtHE975PUId',
                   'EmEtaBCD',
                   'EgBdtHE975EtaBC',
                   'Em',
                   'PUId',
                   'EgBdtLE995PUIdEtaABC',
                   'EgBdtHE995PUIdEtaBC',
                   'EgBdtHE995PUIdEtaBCD',
                   'EtaABC',
                   'EtaBCD',
                   'EgBdtLE995PUIdEtaBCD',
                   'EmPUIdEtaABC',
                   'EgBdtLE975EtaABC',
                   'PUIdEtaBC',
                   'EgBdtHE975',
                   'EgBdtLE995PUId',
                   'EmPUIdEtaBCD',
                   'EgBdtHE995EtaBC',
                   'EmEtaBC',
                   'EgBdtLE995PUIdEtaBC',
                   'EgBdtHE995PUId',
                   'EgBdtLE975EtaBCD',
                   'PUIdEtaABC',
                   'EgBdtHE995PUIdEtaABC',
                   'EgBdtLE975EtaBC',
                   'EgBdtLE975PUIdEtaBC',
                   'EgBdtLE995EtaBC',
                   'EgBdtHE995EtaABC',
                   'EgBdtLE975PUId',
                   'all',
                   'EgBdtLE995EtaABC',
                   'EgBdtLE975PUIdEtaABC',
                   'EgBdtHE975EtaABC',
                   'EmPUIdEtaBC',
                   'EgBdtLE995EtaBCD',
                   'EgBdtLE975PUIdEtaBCD',
                   'EgBdtHE975EtaBCD',
                   'EgBdtHE995EtaBCD',
                   'EgBdtHE975PUIdEtaBC',
                   'EgBdtLE995',
                   'EgBdtHE975PUIdEtaABC',
                   'EgBdtHE995']}

SyntaxError: unmatched '}' (3813848700.py, line 48)

In [59]:
hplot.labels_dict['EgBdtLE995EtaBCD']

'BDT^{eg}_{LE}@995%, 1.52 < |#eta^{L1}| <= 2.8'

In [60]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-hgc'], 'PU200', ['HGCCl3d'], ['EgBdtLE975EtaBC', 'EtaBC', 'EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000


In [61]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-hgc'], 'PU200', ['HGCCl3d'], ['EgBdtLE975EtaBC', 'EtaBC', 'EgBdtLE975PUIdEtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000


In [62]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-hgc'], 'PU200', ['HGCCl3d'], ['EgBdtLE975PUIdEtaABC', 'EgBdtLE995PUIdEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


hsets, labels, text = hplot.get_histo(histos.RateHistos, ['quality'], 'PU200', ['EGStaEE'], ['EtaABC', 'IDTightSEtaABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


In [63]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-hgc'], 'PU200', ['HGCCl3d'], ['EgBdtLE995PUIdEtaBC', 'EgBdtLE975PUIdEtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
   do_ratio=True)




normalize to 31000


In [64]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12.3-tp', '12.3-hgc'], 'PU200', ['HGCCl3d', 'TkEmEE'], ['EmEtaBC', 'EmPUIdEtaBC', 'EgBdtLE975EtaBC', 'EgBdtLE995EtaBC','EgBdtLE995PUIdEtaBC', 'EtaBC', 'EGq1EtaBC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)

# dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=80000,
    x_min=0, x_max=60,
    y_log=True, x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20, 100, 1000],
   do_ratio=True)




normalize to 31000
normalize to 31000
normalize to 31000


In [ ]:
a

In [4]:
import rates_125 as menu

In [ ]:
       ('TkEleL2', 'EtaABC'),
               ('TkEleL2', 'IDTightEEtaABC'),
               ('TkEleL2', 'EtaF'),
               ('TkEleL2', 'IDTightEEtaF'),
               ('TkEleL2', 'EtaFABC'),
               ('TkEleL2', 'IDTightEEtaFABC'),
]
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5'], 'PU200', ['TkEleEE','TkEleL2'], ['IDTightEEtaABC'], None)


In [43]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEleL2'], ['IDTightEEtaFABC'], None)
hsets[0].h_pt_graph(
    'online', 
    'rate [kHz]', 
    convertRateToGraph)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0., y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)





***Warning: Ratio plot can not be added: list index out of range


In [61]:
hsets, labels, text_tkEle = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEleL2'], ['IDTightEEtaFABC'], None)
hsets[0].h_pt_graph(
    'online', 
    'rate [kHz]', 
    convertRateToGraph)
histo_menu = ROOT.TGraph(len(menu.onl['tkElectron']), menu.onl[obj_menu], menu.onlrate['tkElectron'])

hset_tkEle = [histo_menu, hsets[0].g_pt_online]


hsets, labels, text_tkEm = hplot.get_histo(histos.RateHistos, ['12p5new'], 'PU200', ['TkEmL2'], ['EtaFABC'], None)
hsets[0].h_pt_graph(
    'online', 
    'rate [kHz]', 
    convertRateToGraph)

histo_menu = ROOT.TGraph(len(menu.onl['standaloneElectron']), menu.onl[obj_menu], menu.onlrate['standaloneElectron'])

hset_tkEm = [histo_menu, hsets[0].g_pt_online]

labels_menu = ['menu', 'e/g']


normalize to 31000


In [62]:
objs = [('tkElectron', 'TkEleL2'), ('standaloneElectron', 'TkEmL2')]
menu.onl.keys()




(obj_menu, obj_eg) = objs[0]

thrs = menu.onl[obj_menu]
histo = hsets[0].g_pt_online

histo_menu = ROOT.TGraph(len(menu.onl[obj_menu]), menu.onl[obj_menu], menu.onlrate[obj_menu])

# print(thrs)
# range_min = 0
# for (idx, thr) in enumerate(thrs):
#     rate = menu.onlrate[obj_menu][idx]
# #     print(idx, thr, rate )
#     for ix in range(range_min, histo.GetN()):
#         if thr == histo.GetX()[ix]:
# #             print ('   ',thr, histo.GetY()[ix])




In [63]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)
dm.addHistos(hset_tkEle, labels=labels_menu)


dm.addRatioHisto(1,0)

dm.draw(
    text=text_tkEle,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0., y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


***Warning: Ratio plot can not be added: [RatioPlot] num and den TGraphAsymmErrors objs have different # of points: (100 and 30 resp.)


In [64]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)
dm.addHistos(hset_tkEm, labels=labels_menu)


# dm.addRatioHisto(1,0)

dm.draw(
    text=text_tkEm,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0., y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=False)


In [ ]:
rleg_config.legend_size=(0.5, 0.3)
rleg_config.legend_position=(0.3, 0.5)
rleg_config.marker_size = 0.6

dm = DrawMachine(rleg_config)

hsets, labels, text = hplot.get_histo(histos.RateHistos, ['12p3-newID', '12p5new'], 'PU200', ['TkEmL2'], ['IDTightPEtaFABC'], None)
dm.addHistos([his.h_pt for his in hsets], labels=labels)


dm.addRatioHisto(1,0)

dm.draw(
    text=text,
    y_min=1, y_max=40000,
    x_min=0, x_max=100,
    y_min_ratio=0.8, y_max_ratio=2.0,
    y_log=True, 
    x_axis_label='online p_{T} thresh. [GeV]',
    h_lines=[20,100,1000],
   do_ratio=True)


dm.toWeb(name='hRate_TkEmL2_TightID_EtaFABC', page_creator=wc)



In [8]:
menu.onlrate

{'gmtTkMuon': array('d', [17768.8, 8492.5, 948.6, 197.8, 71.3, 33.3, 17.6, 12.4, 8.5, 6.7, 4.7, 3.9, 3.0, 2.8, 2.4, 2.1, 2.1, 1.9, 1.7, 1.5, 1.3, 1.2, 1.2, 0.8, 0.8, 0.8]),
 'gmtMuon': array('d', [10060.1, 4379.2, 1109.2, 448.5, 219.4, 134.9, 85.4, 62.3, 49.9, 38.8, 31.8, 26.7, 22.6, 19.2, 16.8, 14.8, 13.7, 12.6, 11.8, 10.9, 9.6, 8.8, 8.3, 8.0, 6.7, 6.5]),
 'gmtMuonEndcap': array('d', [9161.9, 3222.6, 705.7, 279.7, 147.1, 91.5, 59.9, 43.6, 34.1, 26.6, 22.5, 18.7, 15.8, 12.7, 11.0, 10.1, 9.0, 8.3, 7.6, 6.7]),
 'gmtMuonBarrel': array('d', [625.6, 625.6, 235.2, 89.3, 35.7, 17.4, 10.0, 6.4, 4.1, 3.3, 2.6, 1.5, 1.2, 1.0, 0.9, 0.7, 0.7, 0.6, 0.5, 0.5]),
 'gmtMuonOverlap': array('d', [702.6, 702.5, 181.3, 82.2, 37.6, 26.5, 15.7, 12.4, 11.7, 8.9, 6.7, 6.5, 5.6, 5.5, 4.9, 4.1, 4.1, 3.7, 3.7, 3.7]),
 'tkElectron': array('d', [552.8, 309.7, 150.3, 96.8, 66.1, 44.0, 30.8, 23.4, 17.9, 14.3, 11.6, 10.1, 9.0, 7.8, 6.4, 5.5, 4.9, 3.8, 3.3, 2.8, 2.4, 2.3, 1.9, 1.8, 1.6, 1.3, 1.2, 1.1, 1.1, 0.9]),
 'tkI

## Online to offline scaling

In [30]:
# NOTE: this version of the scaling params is clearly obsolete. It is here just to keep track of how to 
# apply online to offline scaling
# version_V9 = 'v113'
scaling_params =  pd.read_csv(r'scaling_params_{}.csv'.format('v142A'))

In [ ]:
scaling_params

In [ ]:
# eval(scaling_params[(scaling_params.pu == 'PU200') & (scaling_params.tp == 'EG') & (scaling_params.tp_sel == 'EGq3EtaBC')].params.iloc[0])

In [ ]:
tps = hplot.data.tp.unique().tolist()

for sample in samples:
    for tp in tps:
        for tp_sel in ['EGq4EtaBC']:
            hsets, labels, text = hplot.get_histo(histos.RateHistos, sample.type, 'PU200', tp, tp_sel, None)
            print(hsets)
            if hsets is None:
                continue
            for his in hsets:
                his.normalize(31000)
                his.h_pt_graph(
                    'online', 
                    'rate [kHz]', 
                    convertRateToGraph)
                his.h_pt_graph(
                    'offline', 
                    'rate [kHz]', 
                    lambda x: convertRateToGraph(
                        x, 
                        eval(scaling_params[
                                (scaling_params['sample'] == 'ele-'+sample.type) &
                                (scaling_params.pu == 'PU200') & 
                                (scaling_params.tp == tp) & 
                                (scaling_params.tp_sel == tp_sel)].params.iloc[0])))

In [ ]:
for sample_scaling, sample_rate, tp, tp_sel, tp_scaling, tp_sel_scaling in [
    ('ele-V9', 'TDR', 'EG', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'EGBRL', 'EtaF', 'EGBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleELBRL', 'EtaF', 'TkEleELBRL', 'allEtaF'),
    ('ele-V9', 'TDR', 'TkEleEL', 'EGq5EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'Iso0p1EtaBC', 'TkEleEL', 'EGq5EtaBC'),
    ('ele-V9', 'TDR', 'TkEleELALL', 'EtaFIso0p1', 'TkEleELBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'EgEE', 'EGq5EtaBC', 'EG', 'EGq5EtaBC'),
    ('dele-NM', 'NM', 'EgEB', 'EtaF', 'EGBRL', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEB', 'EtaF', 'tkEleEB', 'allEtaF'),
    ('dele-NM', 'NM', 'tkEleEE', 'EGq5EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'Iso0p1EtaBC', 'tkEleEE', 'EGq5EtaBC'),
    ('ele-NM', 'TDR', 'tkEle', 'EtaFIso0p1', 'tkEleEB', 'allEtaF'),

]:
    
        
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_rate, 'PU200', tp, tp_sel, None)
    if hsets is None:
        continue
    for his in hsets:
        his.normalize(31000)
        
        his.h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        his.h_pt_graph(
            'offline', 
            'rate [kHz]', 
            lambda x: convertRateToGraph(
                x, 
                eval(scaling_params[
                        (scaling_params['sample'] == sample_scaling) &
                        (scaling_params.pu == 'PU200') & 
                        (scaling_params.tp == tp_scaling) & 
                        (scaling_params.tp_sel == tp_sel_scaling)].params.iloc[0])))

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V11', 'TDR'], 'PU200', ['HMvDRshapeDrCalib', 'HMvDRshapeDrCalibNew'], 'EmEtaBC', None)
draw([his.g_pt_online for his in hsets], labels, text=text,  
     min_y=1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='online p_{T} thresh. [GeV]')


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', 'HMvDRshapeDrCalib', 'EmEtaBC', None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  
     min_y =1, max_y=40000,
     min_x=0, max_x=60,
     logy=True, x_axis_label='offline p_{T} thresh. [GeV]')


## Compute fixed rate WPs

In [67]:
rate_points = [9, 20, 30]

In [68]:
def cutAtRate(histo, rate):
    """
    Determines the X value (pt cut) corresponding to Y value (rate)
    """
    cut = None
    for ix in range(0, histo.GetN()):
        if histo.GetY()[ix] <= rate:
            cut_low = histo.GetX()[ix]
#             rate_low = histo.GetY()[ix]
            cut_high = histo.GetX()[ix-1]
#             rate_high = histo.GetY()[ix-1]
            cut_middle = cut_high+(cut_low-cut_high)/2.
            rate_middle = histo.Eval(cut_middle)
#             print(f'rate_mid: {rate_middle} @ cut_mid: {cut_middle}')
            if rate_middle <= rate:
                cut = cut_high
            else:
                cut = cut_low
#             print(f'rate: {rate} @ pt: {cut}')
            break
    return cut


In [69]:
--- TP selections:
{   'TkEleEB': ['LooseTkID', 'EtaF', 'all', 'EtaFLooseTkID'],
    'TkEleEE': [   'EGq3EtaBC',
                   'EGq1EtaBCD',
                   'EGq3EtaBCD',
                   'EGq1',
                   'EGq1EtaBC',
                   'EGq3'],
    'TkEmEB': ['all', 'EtaFLooseTkID', 'LooseTkID', 'EtaF'],
    'TkEmEE': [   'EGq3EtaBCD',
                  'EGq3',
                  'EGq1EtaBCD',
                  'EGq3EtaBC',
                  'EGq1EtaBC',
                  'EGq1']}

SyntaxError: invalid syntax (2936113466.py, line 1)

In [70]:
import pprint as pp

hps = hplot.data

tps_and_sel = [('PFNFtkEmEE', 'EGq3EtaBC'),
               ('PFNFtkEleEE', 'EGq3EtaBC'),
               ('TkEmEE', 'EGq3EtaBC'),
               ('TkEleEE', 'EGq3EtaBC'),
               ('TkEleEB', 'LooseTkID'),
               ('TkEleEB', 'EtaFLooseTkID'),
               ('TkEmEB', 'all'),
               ('TkEmEB', 'EtaF'),
               ('HGCCl3d', 'EgBdtLE995EtaBC'),
               ('HGCCl3d', 'EgBdtLE975EtaBC'),
               ('HGCCl3d', 'EgBdtLE995PUIdEtaBC'),
               ('HGCCl3d', 'EgBdtLE975PUIdEtaBC')]

    
# smp = samples[0]
for smp in samples:
    wps = {}
    for tp,tp_sel in tps_and_sel:
        print(f'-- TP: {tp}')
        print(f'   sel: {tp_sel}')
        hsets, labels, text = hplot.get_histo(histos.RateHistos, smp.type, 'PU200', tp, tp_sel, None)
        if hsets is None:
            continue
        if tp not in wps.keys():
            wps[tp] = {}

        hsets[0].h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        rate_cuts = {}
        for rate in rate_points:
            ptcut = cutAtRate(hsets[0].g_pt_online, rate)
#                 ptcut_eval = hsets[0].g_pt_online.Eval(rate)
            rate_cuts[rate] = ptcut
            print(f'       rate: {rate}kHz @ pt: {ptcut}GeV')
            wps[tp][tp_sel] = rate_cuts

    pp.pprint(wps)

    import json
    with open('rate_pt_wps_{}.json'.format(smp.version), 'w') as fp:
        json.dump(wps, fp)

-- TP: PFNFtkEmEE
   sel: EGq3EtaBC
No match found for: sample: 12p3 pu: PU200, tp: PFNFtkEmEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: PFNFtkEleEE
   sel: EGq3EtaBC
No match found for: sample: 12p3 pu: PU200, tp: PFNFtkEleEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEmEE
   sel: EGq3EtaBC
No match found for: sample: 12p3 pu: PU200, tp: TkEmEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEE
   sel: EGq3EtaBC
No match found for: sample: 12p3 pu: PU200, tp: TkEleEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEB
   sel: LooseTkID
No match found for: sample: 12p3 pu: PU200, tp: TkEleEB, tp_sel: LooseTkID, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEB
   sel: EtaFLooseTkID
No match found for: sample: 12p3 pu: PU200, tp: TkEleEB, tp_sel: EtaFLooseTkID, gen_se

No match found for: sample: 12p5new pu: PU200, tp: HGCCl3d, tp_sel: EgBdtLE995PUIdEtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: HGCCl3d
   sel: EgBdtLE975PUIdEtaBC
No match found for: sample: 12p5new pu: PU200, tp: HGCCl3d, tp_sel: EgBdtLE975PUIdEtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
{'TkEmEB': {'EtaF': {9: 60.0, 20: 49.0, 30: 43.0},
            'all': {9: 60.0, 20: 49.0, 30: 43.0}}}
-- TP: PFNFtkEmEE
   sel: EGq3EtaBC
No match found for: sample: compID pu: PU200, tp: PFNFtkEmEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: PFNFtkEleEE
   sel: EGq3EtaBC
No match found for: sample: compID pu: PU200, tp: PFNFtkEleEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEmEE
   sel: EGq3EtaBC
No match found for: sample: compID pu: PU200, tp: TkEmEE, tp_sel: EGq3EtaBC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEle

In [71]:
import pprint as pp

hps = hplot.data

tps_and_sel = [('TkEmEE', 'EtaABC'),
               ('TkEmEE', 'IDTightPEtaABC'),
               ('TkEmEB', 'EtaF'),
               ('TkEmEB', 'IDTightPEtaF'),
               ('TkEleEE', 'EtaABC'),
               ('TkEleEE', 'IDTightEEtaABC'),
               ('TkEleEE', 'IDCompWP955EtaABC'),
               ('TkEleEB', 'EtaF'),
               ('TkEleEB', 'IDTightEEtaF'),
               ('TkEmL2', 'EtaABC'),
               ('TkEmL2', 'IDTightPEtaABC'),
               ('TkEmL2', 'EtaF'),
               ('TkEmL2', 'IDTightPEtaF'),
               ('TkEmL2', 'EtaFABC'),
               ('TkEmL2', 'IDTightPEtaFABC'),               
               ('TkEleL2', 'EtaABC'),
               ('TkEleL2', 'IDTightEEtaABC'),
               ('TkEleL2', 'EtaF'),
               ('TkEleL2', 'IDTightEEtaF'),
               ('TkEleL2', 'EtaFABC'),
               ('TkEleL2', 'IDTightEEtaFABC'),
]

# iso_tps_and_sel = []
# for ts in tps_and_sel:
#     for iso in ['Iso0p1', 'Iso0p2', 'Iso0p3']:
#         old_sel = ts[1].split('Eta')
#         new_sel = (ts[0], f'{old_sel[0]}{iso}Eta{old_sel[1]}')
#         iso_tps_and_sel.append(new_sel)
#         print(new_sel)
#         #         new

# tps_and_sel.extend(iso_tps_and_sel)
        
# smp = samples[0]
for smp in samples:
    print(smp.type)
#     if smp.type != 'quality':
#         continue
    wps = {}
    for tp,tp_sel in tps_and_sel:
        print(f'-- TP: {tp}')
        print(f'   sel: {tp_sel}')
        hsets, labels, text = hplot.get_histo(histos.RateHistos, smp.type, 'PU200', tp, tp_sel, None)
        if hsets is None:
            continue
        if tp not in wps.keys():
            wps[tp] = {}

        hsets[0].h_pt_graph(
            'online', 
            'rate [kHz]', 
            convertRateToGraph)
        rate_cuts = {}
        for rate in rate_points:
            ptcut = cutAtRate(hsets[0].g_pt_online, rate)
#                 ptcut_eval = hsets[0].g_pt_online.Eval(rate)
            rate_cuts[rate] = ptcut
            print(f'       rate: {rate}kHz @ pt: {ptcut}GeV')
            wps[tp][tp_sel] = rate_cuts

    pp.pprint(wps)

    import json
    with open('rate_pt_wps_{}.json'.format(smp.version), 'w') as fp:
        json.dump(wps, fp)

12p3
-- TP: TkEmEE
   sel: EtaABC
No match found for: sample: 12p3 pu: PU200, tp: TkEmEE, tp_sel: EtaABC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEmEE
   sel: IDTightPEtaABC
No match found for: sample: 12p3 pu: PU200, tp: TkEmEE, tp_sel: IDTightPEtaABC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEmEB
   sel: EtaF
       rate: 9kHz @ pt: 62.0GeV
       rate: 20kHz @ pt: 48.0GeV
       rate: 30kHz @ pt: 41.0GeV
-- TP: TkEmEB
   sel: IDTightPEtaF
No match found for: sample: 12p3 pu: PU200, tp: TkEmEB, tp_sel: IDTightPEtaF, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEE
   sel: EtaABC
No match found for: sample: 12p3 pu: PU200, tp: TkEleEE, tp_sel: EtaABC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos'>
-- TP: TkEleEE
   sel: IDTightEEtaABC
No match found for: sample: 12p3 pu: PU200, tp: TkEleEE, tp_sel: IDTightEEtaABC, gen_sel: None, classtype: <class 'python.l1THistos.RateHistos

       rate: 9kHz @ pt: 64.0GeV
       rate: 20kHz @ pt: 52.0GeV
       rate: 30kHz @ pt: 45.0GeV
-- TP: TkEmL2
   sel: IDTightPEtaFABC
       rate: 9kHz @ pt: 45.0GeV
       rate: 20kHz @ pt: 37.0GeV
       rate: 30kHz @ pt: 33.0GeV
-- TP: TkEleL2
   sel: EtaABC
       rate: 9kHz @ pt: 39.0GeV
       rate: 20kHz @ pt: 31.0GeV
       rate: 30kHz @ pt: 25.0GeV
-- TP: TkEleL2
   sel: IDTightEEtaABC
       rate: 9kHz @ pt: 34.0GeV
       rate: 20kHz @ pt: 25.0GeV
       rate: 30kHz @ pt: 20.0GeV
-- TP: TkEleL2
   sel: EtaF
       rate: 9kHz @ pt: 47.0GeV
       rate: 20kHz @ pt: 35.0GeV
       rate: 30kHz @ pt: 31.0GeV
-- TP: TkEleL2
   sel: IDTightEEtaF
       rate: 9kHz @ pt: 34.0GeV
       rate: 20kHz @ pt: 26.0GeV
       rate: 30kHz @ pt: 23.0GeV
-- TP: TkEleL2
   sel: EtaFABC
       rate: 9kHz @ pt: 53.0GeV
       rate: 20kHz @ pt: 41.0GeV
       rate: 30kHz @ pt: 35.0GeV
-- TP: TkEleL2
   sel: IDTightEEtaFABC
       rate: 9kHz @ pt: 43.0GeV
       rate: 20kHz @ pt: 33.0GeV
       ra

       rate: 9kHz @ pt: 37.0GeV
       rate: 20kHz @ pt: 28.0GeV
       rate: 30kHz @ pt: 25.0GeV
-- TP: TkEleL2
   sel: EtaFABC
       rate: 9kHz @ pt: 53.0GeV
       rate: 20kHz @ pt: 43.0GeV
       rate: 30kHz @ pt: 38.0GeV
-- TP: TkEleL2
   sel: IDTightEEtaFABC
       rate: 9kHz @ pt: 44.0GeV
       rate: 20kHz @ pt: 33.0GeV
       rate: 30kHz @ pt: 29.0GeV
{'TkEleEB': {'EtaF': {9: 47.0, 20: 37.0, 30: 32.0},
             'IDTightEEtaF': {9: 37.0, 20: 28.0, 30: 25.0}},
 'TkEleEE': {'EtaABC': {9: 40.0, 20: 32.0, 30: 29.0},
             'IDTightEEtaABC': {9: 32.0, 20: 23.0, 30: 19.0}},
 'TkEleL2': {'EtaABC': {9: 40.0, 20: 32.0, 30: 29.0},
             'EtaF': {9: 47.0, 20: 37.0, 30: 32.0},
             'EtaFABC': {9: 53.0, 20: 43.0, 30: 38.0},
             'IDTightEEtaABC': {9: 32.0, 20: 23.0, 30: 19.0},
             'IDTightEEtaF': {9: 37.0, 20: 28.0, 30: 25.0},
             'IDTightEEtaFABC': {9: 44.0, 20: 33.0, 30: 29.0}},
 'TkEmEB': {'EtaF': {9: 57.0, 20: 47.0, 30: 42.0},
        

In [72]:
dir(samples[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build_file_primitive_index',
 'build_file_primitive_index_oldStyle',
 'histo_file',
 'histo_filename',
 'label',
 'name',
 'oldStyle',
 'open_file',
 'print_file_primitive_index',
 'print_primitives',
 'type',
 'version']

In [73]:
import tabulate

table = []
for smp in samples:
    with open('rate_pt_wps_{}.json'.format(smp.version)) as f:
        pt_wps = json.load(f)
        for tp,tp_wps in pt_wps.items():
            if len(tp_wps) == 0:
                continue

            for tp_sel, wps in tp_wps.items():
                row = []
                row.append(smp.type)
                row.append(f'{tp}, {tp_sel}')
                for rate in rate_points:
                    row.append(wps[str(rate)])
                table.append(row)

print('\n')

print(tabulate.tabulate(table, headers=[
    'sample', 
    'obj and sel.', 
    f'pT @ {rate_points[0]}kHz [GeV]', 
    f'pT @ {rate_points[1]}kHz [GeV]', 
    f'pT @ {rate_points[2]}kHz [GeV]']))
    
                    




sample      obj and sel.                  pT @ 9kHz [GeV]    pT @ 20kHz [GeV]    pT @ 30kHz [GeV]
----------  --------------------------  -----------------  ------------------  ------------------
12p3        TkEmEB, EtaF                               62                  48                  41
12p3        TkEleEB, EtaF                              54                  40                  34
12p3newID   TkEmEE, EtaABC                             46                  37                  33
12p3newID   TkEmEE, IDTightPEtaABC                     42                  33                  28
12p3newID   TkEmEB, EtaF                               63                  48                  42
12p3newID   TkEmEB, IDTightPEtaF                       34                  29                  27
12p3newID   TkEleEE, EtaABC                            38                  28                  24
12p3newID   TkEleEE, IDTightEEtaABC                    32                  23                  20
12p3newID   TkEleE

In [76]:
import tabulate

rate_points = [9, 20]
table = []
for smp in samples:
    with open('rate_pt_wps_{}.json'.format(smp.version)) as f:
        if smp.type == 'ref':
            continue
        pt_wps = json.load(f)
        for tp,tp_wps in pt_wps.items():
            if not 'TkEleEE' in tp or "EtaFLoose" in tp_wps:
                continue

            for tp_sel, wps in tp_wps.items():
                row = []
                row.append(smp.type)

                #                 print(tp_sel)
#                 print(wps)
                row.append(f'{tp}, {tp_sel}')
                for rate in rate_points:
                    row.append(wps[str(rate)])
                table.append(row)

print('\n')

print(tabulate.tabulate(table, headers=[
    'sample', 
    'obj and sel.', 
    f'pT @ {rate_points[0]}kHz [GeV]', 
    f'pT @ {rate_points[1]}kHz [GeV]']))
    
                    




sample      obj and sel.                  pT @ 9kHz [GeV]    pT @ 20kHz [GeV]
----------  --------------------------  -----------------  ------------------
12p3newID   TkEleEE, EtaABC                            38                  28
12p3newID   TkEleEE, IDTightEEtaABC                    32                  23
12p5        TkEleEE, EtaABC                            39                  31
12p5        TkEleEE, IDTightEEtaABC                    34                  25
12p5new     TkEleEE, EtaABC                            33                  26
12p5new     TkEleEE, IDTightEEtaABC                    27                  21
compID      TkEleEE, EtaABC                            40                  32
compID      TkEleEE, IDTightEEtaABC                    32                  23
compID-new  TkEleEE, EtaABC                            40                  32
compID-new  TkEleEE, IDTightEEtaABC                    33                  26
compID-new  TkEleEE, IDCompWP955EtaABC                 32     

In [60]:
import tabulate

rate_points = [20, 100, 1000]
table = []
for smp in samples:
    with open('rate_pt_wps_{}.json'.format(smp.version)) as f:
        if not 'hgc' in smp.type:
            continue
        pt_wps = json.load(f)
        for tp,tp_wps in pt_wps.items():
            if not 'HGCCl3d' in tp:
                continue

            for tp_sel, wps in tp_wps.items():
                row = []
                row.append(smp.type)

                #                 print(tp_sel)
#                 print(wps)
                row.append(f'{tp}, {tp_sel}')
                for rate in rate_points:
                    row.append(wps[str(rate)])
                table.append(row)

print('\n')

print(tabulate.tabulate(table, headers=[
    'sample', 
    'obj and sel.', 
    f'pT @ {rate_points[0]}kHz [GeV]', 
    f'pT @ {rate_points[1]}kHz [GeV]',
f'pT @ {rate_points[2]}kHz [GeV]']))
    
                    




sample    obj and sel.    pT @ 20kHz [GeV]    pT @ 100kHz [GeV]    pT @ 1000kHz [GeV]
--------  --------------  ------------------  -------------------  --------------------


In [61]:
import tabulate
tp = 'TkEleEE'
iso_vars = []
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


NameError: name 'iso_pt_wps' is not defined

In [62]:
import tabulate
tp = 'PFNFtkEmEB'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


NameError: name 'iso_pt_wps' is not defined

In [63]:
import pprint as pp

iso_pt_wps = {}

histo_primitives = hplot.data

tp_sels = [
    ('PFNFtkEmEB', histo_primitives[histo_primitives.tp == 'PFNFtkEmEB'].tp_sel.unique().tolist()),
    ('PFNFtkEmEE', [sel for sel in histo_primitives[histo_primitives.tp == 'PFNFtkEmEE'].tp_sel.unique().tolist() if 'EtaBC' not in sel]),   
]
    
# print (tp_sels)

for tp, tp_sel_list in tp_sels:
    iso_pt_wps[tp] = {}
    for sel in tp_sel_list:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, ['ref'], 'PU200', tp, sel, None)
        
        for his in hsets:
            his.normalize(31000)
        rate_cuts = {}
        for rate in rate_points:
            rate_cuts[rate] = cutAtRate(hsets[0].h_pt, rate)
        iso_pt_wps[tp][sel] = rate_cuts

pp.pprint(iso_pt_wps)

import json
with open('iso_pt_wps.json', 'w') as fp:
    json.dump(iso_pt_wps, fp)

{'PFNFtkEmEB': {}, 'PFNFtkEmEE': {}}


In [ ]:
import tabulate
tp = 'PFNFtkEmEB'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'{iso_var}WP{iso_wp}Pt{pt_wp}'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['all'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')


In [ ]:
import tabulate
tp = 'PFNFtkEmEE'
iso_vars = ['tkIso', 'tkIsoPV', 'pfIso', 'pfIsoPV']
rate = 1000
# iso_wp = 98

for rate in [20, 100, 1000]:
    table = []
    for pt_wp in [10, 20, 40]:
        for iso_wp in [95, 98]:
            row = []
            row.append(f'WP {iso_wp} @ pT > {pt_wp} GeV')
            for iso_var in iso_vars:
                iso_pt_wp = f'EGq5{iso_var}WP{iso_wp}Pt{pt_wp}EtaABC'
                row.append(iso_pt_wps[tp][iso_pt_wp][rate])
            row.append(iso_pt_wps[tp]['EGq5EtaABC'][rate])
            table.append(row)
    print(tabulate.tabulate(table, headers=[f'pT threshold [GeV] @ {rate}kHz']+iso_vars+['no iso.']))
    print('\n')
